# Lab 5: Adversarial Examples and Special Topics

In [1]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Advanced optimizers
SGD with momentum is a fairly good optimizer, but assumes that every parameter it's updating should have the same learning rate (and that the learning rate should stay the same over time).
But, if a parameter contributes to a unit that activates often, it will more frequently receive strong gradient updates, and so have a higher "effective learning rate".
Having parameters with very different "effective learning rates" slows down learning, since some 

Several advanced "adaptive" optimizers have been developed to deal with this problem.
They have the additional advantage of working well with a broader range of learning rates than SGD, so they take less tuning.
They're particularly effective when you expect certain features to be rare or sparse, since this is when differences in effective learning rate appear.

I'll only describe the basics of these algorithms here.
If you're interested in reading more, check out [this post on ruder.io](http://ruder.io/optimizing-gradient-descent/index.html).
The [Keras documentation on optimizers](https://keras.io/optimizers/) also has some good info.

### Adagrad
Adagrad is an adaptive variant of SGD that divides the learning rate applied to a given parameter by the square root of the running total of gradient updates that parameter has _ever_ seen.
As training progresses, the learning rate of each parameter monotonically decreases as it experiences more gradient updates, and the parameters that are updated the most see the sharpest decrease.

Adagrad has the very appealing property that you basically don't need to tune the learning rate; the Keras documentation recommends leaving it fixed at 0.01.
But this robustness comes with a major downside: the "running total of past gradients" it keeps track of per-parameter strictly grows, so eventually the learning rate becomes very small.

This makes Adagrad a bad choice for long-running training tasks.
Qualitatively, I don't see Adagrad used very often.

To use Adagrad in TensorFlow, use [`tf.train.AdagradOptimizer`](https://www.tensorflow.org/api_docs/python/tf/train/AdagradOptimizer).

### RMSProp and Adadelta
Instead of reducing the learning rate for a parameter based on all of the updates it's ever gotten, RMSProp and Adadelta use an exponential moving average of previous gradients, so the reduction is based on only the previous few updates. 
This prevents Adagrad's problem of monotonically-decreasing gradients, but does make it more sensitive to learning rate tuning. 

Of the two, RMSProp seems to be more popular, though there's not a lot of difference.
RMSProp in particular is a very commonly-used optimizer.

To use them, see [`tf.train.RMSPropOptimizer`](https://www.tensorflow.org/api_docs/python/tf/train/RMSPropOptimizer) and [`tf.train.AdadeltaOptimizer`](https://www.tensorflow.org/api_docs/python/tf/train/AdadeltaOptimizer).

### Adam
Adam is a modification of RMSProp with an extra term similar to momentum (using a moving average of past gradients instead of the current gradient).
It's extremely popular, and usually trains your model the fastest with the least tuning.

To use it, see [`tf.train.AdamOptimizer`](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer).

### Simple SGD generalizes better?
Adaptive methods like Adam almost always converge faster much faster, but there is [some evidence that SGD + momentum](http://ruder.io/deep-learning-optimization-2017/index.html#improvingadam) generalizes better.
Generally I stick to Adam, but when you need to squeeze accuracy out of a model (e.g. Kaggle) consider SGD + momentum instead of an adaptive method.

## Flow control in TensorFlow
Generally, it's best to design a TensorFlow graph so that data always flows through the same branch in the same way.
Ideally, each tensor represents exactly one thing.
Because of this, flow control in TensorFlow is pretty awkward.
But there are some cases where, even in graph programming, it's helpful to have conditionals and looping.

### `tf.cond` 
[`tf.cond`](https://www.tensorflow.org/api_docs/python/tf/cond) is the closest thing TensorFlow has to an if-statement.
It takes a scalar ("falsy" values like False and 0 map to False, everything else maps to True) and two _functions that return tensors_. 
Both functions must have the same number and type of outputs.

Its behavior is counterintuitive.
Both "branches" are functions (usually lambdas) that return tensors, and every "upstream" tensor that _either_ branch depends on is evaluated.
Also, both functions get called exactly once, when `tf.cond()` is called, and after that their returned tensors are used instead.

`tf.cond` is useful when part of the graph can take one of multiple similar inputs in different cases.
An example is the assignment, where we use `tf.cond` to switch between an image and that image with added noise.

The syntax is best understood with an example:

In [4]:
a = tf.constant(1.0)
b = tf.constant(2.0)
switch = tf.placeholder(tf.bool, shape=())

# This is one way to pass a tensor to `tf.cond`
def a_fn():
    return a

result = tf.cond(switch, 
                 a_fn, 
                 lambda: b) # Lambdas are the more common way to pass tensors to cond

with tf.Session() as sess:
    print('True branch: result =', sess.run(result, feed_dict={switch: True}))
    print('False branch: result =', sess.run(result, feed_dict={switch: False}))

True branch: result = 1.0
False branch: result = 2.0


### `tf.while_loop`
[`tf.while_loop`](https://www.tensorflow.org/api_docs/python/tf/while_loop) updates a fixed list of tensors, called `loop_vars`, by calling a body function until a condition function is no longer true.

`tf.while_loop` is useful for:
 - Updating values until they meet some condition
 - Running the same (side-effect-ful) operation many times

Again, an example (taken from the documentation) is helpful to understand the syntax:

In [25]:
i = tf.constant(0)

def condition(i):
    return tf.less(i, 10)  # Note: this is a tensor
    
def body(i):
    return tf.add(i, 1)    # Note: this is also a tensor

result = tf.while_loop(cond=condition, 
                       body=body, 
                       loop_vars=[i])  # The variables that get updated

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(result))

10


## Controlling gradient flow in TensorFlow
The way we've used optimizers so far, they just modify every variable in the graph every time they perform a gradient update.
For more complicated models (e.g. with multiple training phases), that behavior might not be desirable.
So, TensorFlow provides a number of ways to control how gradients flow backwards through the graph and which variables are updated.

#### [`tf.stop_gradient`](https://www.tensorflow.org/api_docs/python/tf/stop_gradient)
If `a` is a tensor, then `tf.stop_gradient(a)` is that same tensor, except that _gradients will not flow backwards through this operation_.

Recall from the discussion of backpropagation how the gradient applied to a parameter is the sum of gradients along all paths from that parameter to the output.
If there is a `stop_gradient` operation on one of those paths, that path is removed from the computation.

#### Passing variables to [`minimize()`](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer#minimize)
When calling the `minimize()` function of an optimizer, you can optionally specify `var_list`, a list of variables to be changed when that operation is evaluated.
If `var_list` is None, then every variable will be changed.

#### [Manually modifying gradients](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer#processing_gradients_before_applying_them)
We saw before that an optimizer's `compute_gradients()` and `apply_gradients()` functions allow us access the gradients directly.
If you remove gradients and variables from the list returned by `compute_gradients()` before passing it to `apply_gradients()`, those variables will not be updated.

## Keras functional API
The Keras sequential API works for models that take a single input and produce a single output by applying one layer at a time to that input.
More sophisticated models, though, might:
 - Have more than one input or output
 - Use the output of a single layer in multiple places
 - Share the weights of a single layer in multiple places
 - Take an existing model and adds layers to it
To do any of these things with Keras, you need the functional API.

The best resource for learning the Keras functional API is [the official guide](https://keras.io/getting-started/functional-api-guide/), but the essence is:
 - Layers are functions, not objects
 - When a layer is called on a tensor, it returns a tensor
 - A `Model` object takes an input tensor and an output tensor and learns (using its interior layers and data) to predict the output tensor from the input tensor
 
The Keras functional API seems to me to be the library that mimics how I think about differentiable programming most closely, so it's my favorite way of writing models.
For many kinds of advanced models, it's probably the fastest.

For a full worked example with the functional API, see the bottom of these notes.

## Keras backend
Keras is designed to work with multiple backends, with TensorFlow having the most support. 
Keras' `backend` module is a huge set of wrapper functions that (for the most part) call equivalents in whatever backend you're using and handle things like backpropagation and updating variables.

If you're writing a model where Keras provides most, but not all, of the functionality, a good solution can be making custom layers and loss functions (using the Keras backend) rather than switching to TensorFlow.
Keras layers are implemented as classes, which specify:
 - How their output shape depends on their input shape
 - What variables ("weights") they have
 - How they create output from input, as a small dataflow graph written using the Keras baackend
 

Keras has an [official guide on writing layers](https://keras.io/layers/writing-your-own-keras-layers/), which you should read if you're interested.
The [source code for built-in layers](https://github.com/keras-team/keras/tree/master/keras/layers) is also a good example of how to do this.


## Adversarial examples
![adversarial panda](https://blog.openai.com/content/images/2017/02/adversarial_img_1.png)
(Image source: [OpenAI post on adversarial example research](https://blog.openai.com/adversarial-example-research/))

An "adversarial example" is an input that seems close to "normal" but has added "adversarial noise" designed specifically to trick a model.
Adversarial noise is "trained" through gradient descent, given the weights of a particular model.

It's derived from a simple observation.
Traditionally (within a single batch) we hold the image input to a model fixed, and change the parameters of the model to minimize some quantity.
But, our whole model is differentiable, so there's nothing stopping us from instead holding the weights of the model constant and changing the input image -- backpropagation lets us compute the gradient of the loss with respect to every pixel of the input.
Adversarial noise is trained by using this trick to change the input image to _maximize_ the loss while holding the network weights fixed.
The noise is kept small (either with clipping or regularization) to ensure that the input still looks, to the human eye, pretty much how it should.

Properties of adversarial examples, and how to defend against them, are a [topic of current research](https://blog.openai.com/adversarial-example-research/).
In general, we don't know how to stop adversarial attacks on neural networks.

It might seem like, given the description above, that you need all of the weights of a model downloaded to create adversarial examples for it.
But, as you'll find out in this week's assignment, it's actually sufficient to just be able to provide the model input and see output.

# Example: adversarial examples with the Keras functional API

In [ ]:
# TODO